In [55]:
import geopandas as gpd
import pandas as pd
import os, time

### step 1: prepare Overburden Communities spatial data

In [ ]:
# source data

# Bay Area counties shapefile (unclipped version to be consistent with the Overburdened Communities shapefile)
BayareaCounties_DIR = r"M:\Data\GIS layers\Counties\bay_counties.shp"
# Overburdened Communities shapefile
Overburden_DIR = r"M:\Application\PBA50Plus_Data_Processing\OverburdenedCommunities_analysis\PBA50plus\spatial_data\baaqmd_overburdened_communities_2022.geojson"
# Travel model network input shapefile
# e.g. links_shape_DIR = r"M:\Application\Model One\RTP2025\INPUT_DEVELOPMENT\Networks\BlueprintNetworks_v35\net_2050_Blueprint\shapefiles\network_links.shp"

In [57]:
# load overburdened communities shapefile and dissolve to county
Overburden_gpd = gpd.read_file(Overburden_DIR)
display(Overburden_gpd.head())
Overburden_gpd_dissolve = Overburden_gpd[['County','Acres', 'geometry']]
Overburden_gpd_dissolve['County'] = Overburden_gpd_dissolve['County'].apply(lambda x: x.replace(' County', ''))
Overburden_gpd_dissolve = Overburden_gpd_dissolve.dissolve(by='County', aggfunc='sum')
Overburden_gpd_dissolve = Overburden_gpd_dissolve.reset_index()
# add a field 'COUNTY_OBC' to identify whether overburdened
LOOKUP_COUNTY = pd.DataFrame({
    "county_census": ["001", "013", "041", "055", "075", "081", "085", "095", "097"],
    "County": ["Alameda", "Contra Costa", "Marin", "Napa", "San Francisco", "San Mateo", "Santa Clara", "Solano", "Sonoma"]
})
Overburden_gpd_dissolve = Overburden_gpd_dissolve.merge(LOOKUP_COUNTY, on=['County'], how='left')
Overburden_gpd_dissolve['COUNTY_OBC'] = Overburden_gpd_dissolve['county_census']+'_Overburdened'
display(Overburden_gpd_dissolve)

,County,Acres,MERGE_SRC,Tract,ZIP,Population,CIscore,CIscoreP,Ozone,Ozone_Pctl,...,Elderly_65,Hispanic,White,African_Am,Native_Ame,Asian_Amer,Pacific_Is,Other_Mult,ApproxLoc,geometry
0,Alameda County,3602.121227,CES4_BAAQMD_Top30_1000ft_buffer,NaN,None,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,"POLYGON ((579304.309 4169399.300, 579313.518 4..."
1,Alameda County,421.181924,CES4_BAAQMD_Top30_1000ft_buffer,NaN,None,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,"POLYGON ((580858.161 4170932.665, 580866.229 4..."
2,Alameda County,0.052156,CES4_BAAQMD_Top30_1000ft_buffer,NaN,None,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,"POLYGON ((571463.586 4176385.968, 571607.846 4..."
3,Alameda County,0.017112,CES4_BAAQMD_Top30_1000ft_buffer,NaN,None,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,"POLYGON ((572294.970 4176583.994, 572304.904 4..."
4,Alameda County,0.200896,CES4_BAAQMD_Top30_1000ft_buffer,NaN,None,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,"POLYGON ((571668.158 4176502.620, 571750.271 4..."


c:\ProgramData\Anaconda3\envs\geo_evn310\lib\site-packages\geopandas\geodataframe.py:1543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


,County,geometry,Acres,county_census,COUNTY_OBC
0,Alameda,"MULTIPOLYGON (((558910.259 4182286.290, 558859...",43199.272887,001,001_Overburdened
1,Contra Costa,"MULTIPOLYGON (((560357.571 4194632.540, 560040...",80776.795770,013,013_Overburdened
2,Marin,"MULTIPOLYGON (((550858.885 4189510.442, 550852...",2325.832268,041,041_Overburdened
3,Napa,"MULTIPOLYGON (((562034.943 4223568.718, 562050...",950.730762,055,055_Overburdened
4,San Francisco,"MULTIPOLYGON (((552467.560 4173620.357, 552467...",6552.008210,075,075_Overburdened
5,San Mateo,"MULTIPOLYGON (((560936.560 4158466.149, 560946...",10221.136205,081,081_Overburdened
6,Santa Clara,"MULTIPOLYGON (((603282.131 4128689.939, 603289...",39853.841743,085,085_Overburdened
7,Solano,"MULTIPOLYGON (((555398.858 4221425.380, 552820...",183916.887402,095,095_Overburdened
8,Sonoma,"MULTIPOLYGON (((552191.567 4222399.545, 552176...",8053.161606,097,097_Overburdened


In [ ]:
# load Bay Area counties shapefile
counties_gpd = gpd.read_file(BayareaCounties_DIR)
display(counties_gpd)

# get areas outside of the overburdened communities
non_Overburdened = gpd.overlay(counties_gpd, Overburden_gpd_dissolve, how='difference')
non_Overburdened.rename(columns={'NAME': 'County'}, inplace=True)
# use county id for field 'COUNTY_OBC' to be consistent with an subsequent script
non_Overburdened['COUNTY_OBC'] = non_Overburdened['GEOID'].apply(lambda x: x[2:])
display(non_Overburdened)

# put together
Overburden_all_gpd = pd.concat([Overburden_gpd_dissolve, non_Overburdened], ignore_index=True)

# write out the file for QAQC
Overburden_all_gpd.drop(columns=['GEOID'], inplace=True)
Overburden_all_gpd_DIR = r"M:\Application\PBA50Plus_Data_Processing\OverburdenedCommunities_analysis\PBA50plus\spatial_data\overburdened_communities_all.shp"
Overburden_all_gpd.to_file(Overburden_all_gpd_DIR)


,GEOID,NAME,geometry
0,06097,Sonoma,"POLYGON ((505677.166 4240651.574, 505674.283 4..."
1,06075,San Francisco,"MULTIPOLYGON (((548043.614 4188095.410, 548789..."
2,06041,Marin,"POLYGON ((518783.065 4193293.177, 518725.963 4..."
3,06055,Napa,"POLYGON ((546614.872 4284199.845, 546615.561 4..."
4,06095,Solano,"POLYGON ((581757.266 4241280.771, 581729.692 4..."
5,06013,Contra Costa,"POLYGON ((564382.937 4195443.795, 564368.696 4..."
6,06085,Santa Clara,"POLYGON ((584827.591 4117542.151, 584830.219 4..."
7,06001,Alameda,"POLYGON ((563387.415 4173575.695, 563307.232 4..."
8,06081,San Mateo,"POLYGON ((536452.983 4160134.450, 536480.048 4..."


C:\Users\ywang\AppData\Local\Temp\ipykernel_9276\1090159493.py:6: UserWarning: `keep_geom_type=True` in overlay resulted in 19 dropped geometries of different geometry types than df1 has. Set `keep_geom_type=False` to retain all geometries
  non_Overburdened = gpd.overlay(counties_gpd, Overburden_gpd_dissolve, how='difference')


,GEOID,County,geometry,COUNTY_OBC
0,06097,Sonoma,"MULTIPOLYGON (((505651.739 4240632.916, 505607...",097
1,06075,San Francisco,"MULTIPOLYGON (((550807.711 4189452.701, 550872...",075
2,06041,Marin,"POLYGON ((518621.121 4193745.487, 518549.565 4...",041
3,06055,Napa,"MULTIPOLYGON (((554297.731 4223589.890, 554287...",055
4,06095,Solano,"MULTIPOLYGON (((622951.713 4241465.567, 622922...",095
5,06013,Contra Costa,"MULTIPOLYGON (((564323.298 4195414.144, 564313...",013
6,06085,Santa Clara,"MULTIPOLYGON (((584837.832 4117591.517, 584840...",085
7,06001,Alameda,"MULTIPOLYGON (((563002.964 4174282.866, 562920...",001
8,06081,San Mateo,"MULTIPOLYGON (((536723.529 4161443.601, 536806...",081


C:\Users\ywang\AppData\Local\Temp\ipykernel_9276\1090159493.py:18: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  Overburden_all_gpd.to_file(Overburden_all_gpd_DIR)


### step 2: create link to Overburdened Communities crosswalk

In [ ]:
%run X:\travel-model-one-master\utilities\cube-to-shapefile\correspond_link_to_TAZ.py "M:\Application\Model One\RTP2025\INPUT_DEVELOPMENT\Networks\BlueprintNetworks_v35\net_2023_Baseline\shapefiles\network_links.shp" "M:\Application\Model One\RTP2025\INPUT_DEVELOPMENT\Networks\BlueprintNetworks_v35\net_2023_Baseline\shapefiles\link_to_COUNTY_Overburdened.csv" --shapefile "M:\Application\PBA50Plus_Data_Processing\OverburdenedCommunities_analysis\PBA50plus\spatial_data\overburdened_communities_all.shp" --shp_id COUNTY_OBC --linkshp_mi linkOBC_mi --linkshp_share linkOBC_share
%run X:\travel-model-one-master\utilities\cube-to-shapefile\correspond_link_to_TAZ.py "M:\Application\Model One\RTP2025\INPUT_DEVELOPMENT\Networks\BlueprintNetworks_v35\net_2050_Baseline\shapefiles\network_links.shp" "M:\Application\Model One\RTP2025\INPUT_DEVELOPMENT\Networks\BlueprintNetworks_v35\net_2050_Baseline\shapefiles\link_to_COUNTY_Overburdened.csv" --shapefile "M:\Application\PBA50Plus_Data_Processing\OverburdenedCommunities_analysis\PBA50plus\spatial_data\overburdened_communities_all.shp" --shp_id COUNTY_OBC --linkshp_mi linkOBC_mi --linkshp_share linkOBC_share
%run X:\travel-model-one-master\utilities\cube-to-shapefile\correspond_link_to_TAZ.py "M:\Application\Model One\RTP2025\INPUT_DEVELOPMENT\Networks\BlueprintNetworks_v35\net_2050_Blueprint\shapefiles\network_links.shp" "M:\Application\Model One\RTP2025\INPUT_DEVELOPMENT\Networks\BlueprintNetworks_v35\net_2050_Blueprint\shapefiles\link_to_COUNTY_Overburdened.csv" --shapefile "M:\Application\PBA50Plus_Data_Processing\OverburdenedCommunities_analysis\PBA50plus\spatial_data\overburdened_communities_all.shp" --shp_id COUNTY_OBC --linkshp_mi linkOBC_mi --linkshp_share linkOBC_share

Read 33,953 links from M:\Application\Model One\RTP2025\INPUT_DEVELOPMENT\Networks\BlueprintNetworks_v35\net_2023_Baseline\shapefiles\network_links.shp
network_links.crs=epsg:26910
network_links.crs=EPSG:26910
sum(network_links[link_mi]) = 4,614.15
Read 18 links from M:\Application\PBA50Plus_Data_Processing\OverburdenedCommunities_analysis\PBA50plus\spatial_data\overburdened_communities_all.shp
After intersecting links with shapefile, have 35,959 rows
check_df for intersect failure: check_df._merge.value_counts()=_merge
both          35959
left_only        60
right_only        0
Name: count, dtype: int64
unjoined_df len=60 type=<class 'geopandas.geodataframe.GeoDataFrame'>
           A      B  DISTANCE  SPDCLASS  CAPCLASS  LANES  TSIN  GL  USE  \
2839    1459   8625      0.01        56        56      7     1  10    1   
2843    1462  11918      0.01        56        56      7     1  10    1   
2845    1463  11896      0.01        56        56      7     1  10    1   
2851    1466   412

### step 3: join links to emission rates

In [ ]:
%run X:\travel-model-one-master\utilities\RTP\Emissions\over_burdened_communities_emission_inventory\join_networklinks_to_emissionrates.py RTP2025

['a', 'b', 'distance', 'lanes', 'gl', 'ft', 'at', 'tollclass', 'state', 'cityid', 'cityname', 'regfreight', 'brt', 'cap', 'ffs', 'fft', 'autoopc', 'smtropc', 'lrtropc', 'busopc', 'cspdEA', 'cspdAM', 'cspdMD', 'cspdPM', 'cspdEV', 'volEA_tot', 'volAM_tot', 'volMD_tot', 'volPM_tot', 'volEV_tot', 'ctimEA', 'ctimAM', 'ctimMD', 'ctimPM', 'ctimEV', 'vcEA', 'vcAM', 'vcMD', 'vcPM', 'vcEV', 'useEA', 'useAM', 'useMD', 'usePM', 'useEV', 'volEA_da', 'volAM_da', 'volMD_da', 'volPM_da', 'volEV_da', 'volEA_s2', 'volAM_s2', 'volMD_s2', 'volPM_s2', 'volEV_s2', 'volEA_s3', 'volAM_s3', 'volMD_s3', 'volPM_s3', 'volEV_s3', 'volEA_sm', 'volAM_sm', 'volMD_sm', 'volPM_sm', 'volEV_sm', 'volEA_hv', 'volAM_hv', 'volMD_hv', 'volPM_hv', 'volEV_hv', 'volEA_dat', 'volAM_dat', 'volMD_dat', 'volPM_dat', 'volEV_dat', 'volEA_s2t', 'volAM_s2t', 'volMD_s2t', 'volPM_s2t', 'volEV_s2t', 'volEA_s3t', 'volAM_s3t', 'volMD_s3t', 'volPM_s3t', 'volEV_s3t', 'volEA_smt', 'volAM_smt', 'volMD_smt', 'volPM_smt', 'volEV_smt', 'volEA_hvt'

In [71]:
df['gl'].unique()

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10], dtype=int64)

In [69]:
display(df[index_cols + special_cols + [col for col in df.columns if col.startswith("cspd")]])
df[index_cols + [col for col in df.columns if col.startswith("cspd")]].melt(id_vars=index_cols, var_name="time_period", value_name="cspd")

,a,b,distance,lanes,ft,at,tollclass,ffs,cspdEA,cspdAM,cspdMD,cspdPM,cspdEV
0,1,7487,0.05,7,6,0,0,18.0,0.00,0.00,0.00,0.00,0.00
1,1,7489,0.05,7,6,0,0,18.0,0.00,0.00,0.00,0.00,0.00
2,2,7496,0.06,7,6,0,0,18.0,0.00,0.00,0.00,0.00,0.00
3,2,7500,0.05,7,6,0,0,18.0,0.00,0.00,0.00,0.00,0.00
4,3,7483,0.13,7,6,0,0,18.0,0.00,0.00,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
34286,39045,15983,0.10,3,3,2,0,45.0,44.99,44.56,44.77,44.50,44.98
34287,39045,39044,0.08,1,7,2,0,20.0,20.00,19.85,19.33,19.93,20.00
34288,39046,5452,0.04,3,3,2,0,45.0,44.99,44.17,44.91,44.45,44.98
34289,39046,15861,0.14,1,7,2,0,20.0,20.00,19.86,19.98,19.99,20.00


,a,b,time_period,cspd
0,1,7487,cspdEA,0.00
1,1,7489,cspdEA,0.00
2,2,7496,cspdEA,0.00
3,2,7500,cspdEA,0.00
4,3,7483,cspdEA,0.00
...,...,...,...,...
171450,39045,15983,cspdEV,44.98
171451,39045,39044,cspdEV,20.00
171452,39046,5452,cspdEV,44.98
171453,39046,15861,cspdEV,20.00
